In [10]:
import pandas as pd

def wide_to_long(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convierte una tabla demográfica en formato ancho (Hombres_0-4, Mujeres_0-4, ...)
    a formato largo con columnas: COD_LOC, NOM_LOC, AÑO, GENERO, RANGO, POBLACION.
    """
    # Columnas clave requeridas
    id_cols = ["COD_LOC", "NOM_LOC", "AÑO"]
    faltantes = [c for c in id_cols if c not in df.columns]
    if faltantes:
        raise ValueError(f"Faltan columnas obligatorias: {faltantes}")

    # Columnas candidatas de población (las que tienen un "_")
    pop_cols = [c for c in df.columns if "_" in str(c)]
    if not pop_cols:
        raise ValueError("No se encontraron columnas de población con el patrón 'Genero_Rango' (p. ej. 'Hombres_0-4').")

    # Reestructurar a formato largo
    m = df.melt(id_vars=id_cols, value_vars=pop_cols, var_name="GEN_RANGO", value_name="POBLACION")

    # Filtrar sólo Hombres_/Mujeres_
    m = m[m["GEN_RANGO"].str.startswith(("Hombres_", "Mujeres_"), na=False)].copy()

    # Separar GENERO y RANGO
    split = m["GEN_RANGO"].str.split("_", n=1, expand=True)
    m["GENERO"] = split[0].map({"Hombres": "Hombre", "Mujeres": "Mujer"}).fillna(split[0])
    m["RANGO"] = split[1].astype(str).str.strip()

    # Limpiar y tipos
    m["COD_LOC"] = m["COD_LOC"].astype(str).str.strip()
    m["NOM_LOC"] = m["NOM_LOC"].astype(str).str.strip()
    m["POBLACION"] = pd.to_numeric(m["POBLACION"], errors="coerce")

    # Orden final
    out = (
        m[["COD_LOC", "NOM_LOC", "AÑO", "GENERO", "RANGO", "POBLACION"]]
        .sort_values(by=["COD_LOC", "AÑO", "GENERO", "RANGO"], kind="stable")
        .reset_index(drop=True)
    )
    return out


In [11]:
url = "https://raw.githubusercontent.com/Sxmuu/TG-Samuel-P/main/Databases/Poblac/Original/Poblacion_Filtrada.xlsx"

df_src = pd.read_excel(url, engine="openpyxl")
print("Columnas originales:", list(df_src.columns))

df_long = wide_to_long(df_src)
print("Filas transformadas:", len(df_long))
df_long.head(10)


Columnas originales: ['COD_LOC', 'NOM_LOC', 'AREA', 'AÑO', 'Hombres_0-4', 'Hombres_5-9', 'Hombres_10-14', 'Hombres_15-19', 'Hombres_20-24', 'Hombres_25-29', 'Hombres_30-34', 'Hombres_35-39', 'Hombres_40-44', 'Hombres_45-49', 'Hombres_50-54', 'Hombres_55-59', 'Hombres_60-64', 'Hombres_65-69', 'Hombres_70-74', 'Hombres_75-79', 'Hombres_80-84', 'Hombres_85-89', 'Hombres_90-94', 'Hombres_95-99', 'Hombres_100+', 'Mujeres_0-4', 'Mujeres_5-9', 'Mujeres_10-14', 'Mujeres_15-19', 'Mujeres_20-24', 'Mujeres_25-29', 'Mujeres_30-34', 'Mujeres_35-39', 'Mujeres_40-44', 'Mujeres_45-49', 'Mujeres_50-54', 'Mujeres_55-59', 'Mujeres_60-64', 'Mujeres_65-69', 'Mujeres_70-74', 'Mujeres_75-79', 'Mujeres_80-84', 'Mujeres_85-89', 'Mujeres_90-94', 'Mujeres_95-99', 'Mujeres_100+']
Filas transformadas: 3360


,COD_LOC,NOM_LOC,AÑO,GENERO,RANGO,POBLACION
0,1,Usaquén,2021,Hombre,0-4,15515
1,1,Usaquén,2021,Hombre,10-14,14434
2,1,Usaquén,2021,Hombre,100+,220
3,1,Usaquén,2021,Hombre,15-19,15747
4,1,Usaquén,2021,Hombre,20-24,21446
5,1,Usaquén,2021,Hombre,25-29,24296
6,1,Usaquén,2021,Hombre,30-34,23452
7,1,Usaquén,2021,Hombre,35-39,22350
8,1,Usaquén,2021,Hombre,40-44,20543
9,1,Usaquén,2021,Hombre,45-49,16636


In [16]:
#Renombra Año a Ano
df_long = df_long.rename(columns={'AÑO': 'ANO'})

In [17]:
df_long

,COD_LOC,NOM_LOC,ANO,GENERO,RANGO,POBLACION
0,1,Usaquén,2021,Hombre,0-4,15515
1,1,Usaquén,2021,Hombre,10-14,14434
2,1,Usaquén,2021,Hombre,100+,220
3,1,Usaquén,2021,Hombre,15-19,15747
4,1,Usaquén,2021,Hombre,20-24,21446
...,...,...,...,...,...,...
3355,9,Fontibón,2024,Mujer,75-79,5063
3356,9,Fontibón,2024,Mujer,80-84,3228
3357,9,Fontibón,2024,Mujer,85-89,1539
3358,9,Fontibón,2024,Mujer,90-94,650


In [18]:
import pandas as pd
import unicodedata

def quitar_tildes(texto):
    if pd.isna(texto):
        return texto
    # Normaliza y elimina marcas diacríticas (Mn)
    return "".join(
        c for c in unicodedata.normalize("NFD", str(texto))
        if unicodedata.category(c) != "Mn"
    )

# Ejemplo: quitar tildes en la columna 'NOM_LOC'
df_long["NOM_LOC"] = df_long["NOM_LOC"].map(quitar_tildes)


In [19]:
df_long

,COD_LOC,NOM_LOC,ANO,GENERO,RANGO,POBLACION
0,1,Usaquen,2021,Hombre,0-4,15515
1,1,Usaquen,2021,Hombre,10-14,14434
2,1,Usaquen,2021,Hombre,100+,220
3,1,Usaquen,2021,Hombre,15-19,15747
4,1,Usaquen,2021,Hombre,20-24,21446
...,...,...,...,...,...,...
3355,9,Fontibon,2024,Mujer,75-79,5063
3356,9,Fontibon,2024,Mujer,80-84,3228
3357,9,Fontibon,2024,Mujer,85-89,1539
3358,9,Fontibon,2024,Mujer,90-94,650


In [20]:
df_long['NOM_LOC'].unique()

array(['Usaquen', 'Engativa', 'Suba', 'Barrios Unidos', 'Teusaquillo',
       'Los Martires', 'Antonio Narino', 'Puente Aranda', 'La Candelaria',
       'Rafael Uribe Uribe', 'Ciudad Bolivar', 'Chapinero', 'Sumapaz',
       'Santa Fe', 'San Cristobal', 'Usme', 'Tunjuelito', 'Bosa',
       'Kennedy', 'Fontibon'], dtype=object)

In [22]:
df_long.to_excel('Poblacion_Transpuesto.xlsx')

In [24]:
#Deja sólo estas NOM_LOC: 'Barrios Unidos', 'Ciudad Bolivar', 'Suba', 'Fontibon','Puente Aranda', 'Kennedy', 'Engativa', 'Santa Fe', 'San Cristobal', 'Tunjuelito', 'Usme'
df_long_filtered = df_long[df_long['NOM_LOC'].isin(['Barrios Unidos', 'Ciudad Bolivar', 'Suba', 'Fontibon','Puente Aranda', 'Kennedy', 'Engativa', 'Santa Fe', 'San Cristobal', 'Tunjuelito', 'Usme'])]

df_long_filtered


,COD_LOC,NOM_LOC,ANO,GENERO,RANGO,POBLACION
168,10,Engativa,2021,Hombre,0-4,20164
169,10,Engativa,2021,Hombre,10-14,23054
170,10,Engativa,2021,Hombre,100+,131
171,10,Engativa,2021,Hombre,15-19,26429
172,10,Engativa,2021,Hombre,20-24,35100
...,...,...,...,...,...,...
3355,9,Fontibon,2024,Mujer,75-79,5063
3356,9,Fontibon,2024,Mujer,80-84,3228
3357,9,Fontibon,2024,Mujer,85-89,1539
3358,9,Fontibon,2024,Mujer,90-94,650


In [25]:
df_long_filtered["NOM_LOC"].unique()

array(['Engativa', 'Suba', 'Barrios Unidos', 'Puente Aranda',
       'Ciudad Bolivar', 'Santa Fe', 'San Cristobal', 'Usme',
       'Tunjuelito', 'Kennedy', 'Fontibon'], dtype=object)

In [26]:
df_long_filtered.to_excel('Poblacion_Transpuesto_Filtrado.xlsx')